In [48]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Importing Data

In [49]:
column_names = ['engine_id', 'cycle', 'setting1', 'setting2', 'setting3'] + [f'sensor{i}' for i in range(1, 22)] # Last 5 columns all NA
train_df = pd.read_csv('./dataset/train_FD001.txt', sep='\s+', header=None, names=column_names)
test_df = pd.read_csv('./dataset/test_FD001.txt', sep='\s+', header=None, names=column_names)
true_rul = pd.read_csv('./dataset/RUL_FD001.txt', header=None)

# Preprocessing Data

In [50]:
# KIV: Should we include settings?
# Dropping columns
columns_to_drop = ["setting1", "setting2", "setting3", "sensor1", "sensor5", "sensor6", "sensor10", "sensor16", "sensor18", "sensor19"]
train_df_dropped = train_df.drop(columns=columns_to_drop)
test_df_dropped = test_df.drop(columns=columns_to_drop)

# Labelling Dataset

In [51]:
# Labelling RUL
# train_df_dropped['RUL'] = train_df_dropped.groupby('engine_id')['cycle'].transform(lambda x: x.max() - x)


# Feature Extraction

In [52]:
# Feature extraction for TSFRESH
from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters
 
# Combine your 30-cycle windowed series with unit IDs as a DataFrame for tsfresh
train_series_df = pd.DataFrame(train_df_dropped)
train_series_df["engine_id"] = train_df_dropped["engine_id"]
# Ensure each unit has a 'time' column cycling from 0 to 29
train_series_df["time"] = train_series_df.groupby("engine_id").cumcount() % 30
 
# Set up tsfresh parameters for comprehensive feature extraction
settings = ComprehensiveFCParameters()


In [53]:
# Extract features using tsfresh
train_features = extract_features(
    train_series_df,
    column_id="engine_id",
    column_sort="time",
    default_fc_parameters=settings,
    n_jobs=4,  # Parallelize if possible to speed up extraction
)

Feature Extraction:   0%|          | 0/20 [00:00<?, ?it/s]

Feature Extraction: 100%|██████████| 20/20 [00:39<00:00,  1.98s/it]


In [59]:
train_features.shape

(100, 11745)

# Scaling of Data

In [55]:
# Normalization
# KIV: Standardization
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
scaler = MinMaxScaler()

# Separate the columns to normalize and the columns to skip
columns_to_skip = train_df_dropped.columns[:2]
columns_to_normalize = train_df_dropped.columns[2:]

# Normalize only the selected columns
normalized_data = scaler.fit_transform(train_df_dropped[columns_to_normalize])

# Combine the normalized and unnormalized columns
train_df_normalized = pd.DataFrame(train_df_dropped[columns_to_skip].values, columns=columns_to_skip)
train_df_normalized = pd.concat([train_df_normalized, pd.DataFrame(normalized_data, columns=columns_to_normalize)], axis=1)

# Display the normalized DataFrame
print("Normalized Data (0-1 range):")
print(train_df_normalized.head())

Normalized Data (0-1 range):
   engine_id  cycle   sensor2   sensor3   sensor4   sensor7   sensor8  \
0          1      1  0.183735  0.406802  0.309757  0.726248  0.242424   
1          1      2  0.283133  0.453019  0.352633  0.628019  0.212121   
2          1      3  0.343373  0.369523  0.370527  0.710145  0.272727   
3          1      4  0.343373  0.256159  0.331195  0.740741  0.318182   
4          1      5  0.349398  0.257467  0.404625  0.668277  0.242424   

    sensor9  sensor11  sensor12  sensor13  sensor14  sensor15  sensor17  \
0  0.109755  0.369048  0.633262  0.205882  0.199608  0.363986  0.333333   
1  0.100242  0.380952  0.765458  0.279412  0.162813  0.411312  0.333333   
2  0.140043  0.250000  0.795309  0.220588  0.171793  0.357445  0.166667   
3  0.124518  0.166667  0.889126  0.294118  0.174889  0.166603  0.333333   
4  0.149960  0.255952  0.746269  0.235294  0.174734  0.402078  0.416667   

   sensor20  sensor21  
0  0.713178  0.724662  
1  0.666667  0.731014  
2  0.6279

In [56]:
# Train, Validate, Test approach





In [57]:
# Data Labelling
# RUL Column
# Label Column
# Setting of early_rul